In [1]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import joblib

# Eingabedaten erzeugen

In [3]:
inputdata = pd.read_pickle("../data/recodata.pkl")

In [4]:
inputdata = pd.DataFrame([[0]*len(inputdata.columns)], columns=inputdata.columns)
inputdata

,yearpublished,playingtime,age,cat_Economic,cat_CardGame,cat_Fantasy,cat_Fighting,cat_ScienceFiction,cat_Adventure,cat_Exploration,...,mec_Ratio/CombatResultsTable,mec_Chaining,mec_Slide/Push,mec_AuctionCompensation,mec_Ordering,mec_PassedActionToken,mec_ElapsedRealTimeEnding,mec_BidsAsWagers,mec_Drawing,mec_PiecesasMap
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#inputdata.columns.tolist()

In [8]:
inputdata["yearpublished"] = 2020
inputdata["playingtime"] = 60
inputdata["age"] = 10
for x in ["cat_CardGame", "cat_ScienceFiction", "cat_Dice", "cat_Animals", "mec_DiceRolling", "mec_ModularBoard"]:
        inputdata[x]=1

In [11]:
inputdata

,yearpublished,playingtime,age,cat_Economic,cat_CardGame,cat_Fantasy,cat_Fighting,cat_ScienceFiction,cat_Adventure,cat_Exploration,...,mec_Ratio/CombatResultsTable,mec_Chaining,mec_Slide/Push,mec_AuctionCompensation,mec_Ordering,mec_PassedActionToken,mec_ElapsedRealTimeEnding,mec_BidsAsWagers,mec_Drawing,mec_PiecesasMap
0,2020,60,10,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


# Eingabedaten transformieren

In [31]:
pca_cat = joblib.load('../data/pca_cat.pkl')
ncomp = pca_cat.n_components_ 
categories_pca = pd.DataFrame(pca_cat.transform(inputdata[[x for x in inputdata.columns if x[:4]=="cat_"]]), columns=["catp_" + str(x) for x in range(ncomp)])

In [36]:
categories_pca

,catp_0,catp_1,catp_2,catp_3,catp_4,catp_5,catp_6,catp_7,catp_8,catp_9,catp_10,catp_11,catp_12,catp_13,catp_14,catp_15,catp_16,catp_17,catp_18,catp_19
0,-0.09279,0.813279,0.357823,0.437768,-0.214335,-0.072403,-0.79354,-0.124019,-0.447716,-0.001893,-0.075348,0.379053,-0.407984,0.440941,-0.067389,-0.28936,-0.058886,0.372242,0.291853,-0.161234


In [32]:
pca_mec = joblib.load('../data/pca_mec.pkl')

In [33]:
ncomp = pca_mec.n_components_ 

In [34]:
mechanic_pca = pd.DataFrame(pca_mec.transform(inputdata[[x for x in inputdata.columns if x[:4]=="mec_"]]), columns=["mecp_" + str(x) for x in range(ncomp)])

In [35]:
mechanic_pca

,mecp_0,mecp_1,mecp_2,mecp_3,mecp_4,mecp_5,mecp_6,mecp_7,mecp_8,mecp_9,mecp_10,mecp_11,mecp_12,mecp_13,mecp_14,mecp_15,mecp_16,mecp_17,mecp_18,mecp_19
0,0.174938,-0.610407,-0.13722,-0.763051,0.112869,-0.08455,-0.093231,-0.139891,0.414312,-0.085435,-0.178328,-0.186738,0.002219,0.407219,0.191645,0.021911,-0.364521,0.09265,-0.165886,0.10114


### StandardScaler laden

In [41]:
allfeat = ['yearpublished', 'playingtime', 'age'] 
for x in allfeat:
    scaler = joblib.load(f'../data/scaler_{x}.pkl')
    inputdata[x] = scaler.transform(inputdata[x].values.reshape(-1, 1))

In [42]:
inputdata

,yearpublished,playingtime,age,cat_Economic,cat_CardGame,cat_Fantasy,cat_Fighting,cat_ScienceFiction,cat_Adventure,cat_Exploration,...,mec_Ratio/CombatResultsTable,mec_Chaining,mec_Slide/Push,mec_AuctionCompensation,mec_Ordering,mec_PassedActionToken,mec_ElapsedRealTimeEnding,mec_BidsAsWagers,mec_Drawing,mec_PiecesasMap
0,-7.105427e-18,-7.105427e-18,3.552714e-17,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
y_test = inputdata[allfeat].copy()

In [45]:
y_test = pd.concat([y_test,categories_pca], axis=1)
y_test = pd.concat([y_test,mechanic_pca], axis=1)

In [46]:
y_test

,yearpublished,playingtime,age,catp_0,catp_1,catp_2,catp_3,catp_4,catp_5,catp_6,...,mecp_10,mecp_11,mecp_12,mecp_13,mecp_14,mecp_15,mecp_16,mecp_17,mecp_18,mecp_19
0,-7.105427e-18,-7.105427e-18,3.552714e-17,-0.09279,0.813279,0.357823,0.437768,-0.214335,-0.072403,-0.79354,...,-0.178328,-0.186738,0.002219,0.407219,0.191645,0.021911,-0.364521,0.09265,-0.165886,0.10114
